# Fluent Python
https://github.com/fluentpython/example-code

파이썬 용어집 

https://docs.python.org/ko/3/glossary.html

In [5]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np

def time_check(func):
    def decorated():
        import time
        start = time.time()
        func()
        print("---{}s seconds---".format(time.time()-start_time))
    return decorated# Fluent Python

## CHAPTER 10 Sequence Hacking, Hashing, and Slicing

What is a Sequence?
A sequence is an ordered collection of objects.

They are analogous to what are often called “arrays” or “lists” in other programming languages.

But in Python, there are number of types that all fit this description, each with special customization. But any object that has the behavior expected of a sequence can be treated the same way in Python:

Remember Duck Typing?

If it looks like a duck and quacks like a duck…

OR: If it looks and acts like a sequence – it is a sequence.

Technically, if it satisfies the “Sequence Protocol”, it is a sequence.

Python is all about these protocols – we will see more of them.

https://uwpce-pythoncert.github.io/PythonCertDevel/modules/Sequences.html


In [5]:
import reprlib
reprlib.repr(list(range(10))) # 너무 길어지면 print 를 ... 처리함 

'[0, 1, 2, 3, 4, 5, ...]'

### Protocols and Duck Typing

In the context of object-oriented programming, a protocol is an informal interface, defined only in documentation and not in code.

어디에도 시퀀스 프로토콜이라고 정의하지 않지만 시퀀스 처럼 동작하기 때문에 시퀀스 인것이다. 

시퀀스 처럼 동작 하기 위해서 \__len__(), \__getitem__() 으로 반복을 지원한다.


### Vector Take #2: A Sliceable Sequence

In [19]:
from array import array
import reprlib
import math


class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)  # <1>

    def __iter__(self):
        return iter(self._components)  # <2>

    def __repr__(self):
        components = reprlib.repr(self._components)  # <3>
        components = components[components.find('['):-1]  # <4>
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))  # <5>

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))  # <6>

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)  # <7>

    # 시퀀스 동작을 위해서 아래 두개 함수를 추가
    
    def __len__(self):
        return len(self._components)

#     def __getitem__(self, index):
#         return self._components[index]
    
    # A Slice-Aware __getitem__ 객체 슬라이스 인식
    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))

In [8]:
v1 = Vector([3, 4, 5])
v1[0]
v1[0:2]

3.0

array('d', [3.0, 4.0])

### How Slicing Works

In [9]:
class MySeq:
    def __getitem__(self, index):
        return index 

In [10]:
s = MySeq()
s[1:4]

slice(1, 4, None)

In [11]:
s[1:4:2]

slice(1, 4, 2)

In [15]:
s[1:4:2, 9] # 콤마가 들어가면 getitem 이 튜플을 받는다

(slice(1, 4, 2), 9)

In [13]:
slice

slice

In [14]:
dir(slice)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'indices',
 'start',
 'step',
 'stop']

In [16]:
help(slice.indices)

Help on method_descriptor:

indices(...)
    S.indices(len) -> (start, stop, stride)
    
    Assuming a sequence of length len, calculate the start and stop
    indices, and the stride length of the extended slice described by
    S. Out of bounds indices are clipped in a manner consistent with the
    handling of normal slices.



In [17]:
slice(None, 10, 2).indices(5) # 'ABCDE'[:10:2] is the same as 'ABCDE'[0:5:2]

(0, 5, 2)

In [18]:
slice(-3, None, None).indices(5) # 'ABCDE'[-3:] is the same as 'ABCDE'[2:5:1]

(2, 5, 1)

### A Slice-Aware __getitem__

In [20]:
v7 = Vector(range(7))
v7[1:4]

Vector([1.0, 2.0, 3.0])

In [21]:
v7[-1:]

Vector([6.0])

In [22]:
v7[1,2]

NameError: name 'numbers' is not defined

### Vector Take #3: Dynamic Attribute Access

In [31]:
class Vector:
    typecode = 'd'
    shortcut_names = 'xyzt'
    
    def __init__(self, components):
        self._components = array(self.typecode, components)  # <1>

    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
        if 0 <= pos < len(self._components):
            return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))
        
    def __repr__(self):
        components = reprlib.repr(self._components)  # <3>
        components = components[components.find('['):-1]  # <4>
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

In [32]:
v = Vector(range(5))
v.x

0.0

In [33]:
v.x = 10
v # the vector components did not change.

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

so we need \__setattr__() 

### Vector Take #4: Hashing and a Faster ==

reduce() : 일련의 값을 하나로 줄인다. 

reduce(fn, list) 호출하면 -> fn(lst[0], lst[1]) 결과를 r1 을 생성하서 저장하고 -> fn(r1, lst[2]) 를 실행한다. 


In [1]:
import functools
functools.reduce(lambda a,b: a*b, range(1,6))

120

In [10]:
#해시xor 연산자 
import operator
functools.reduce(operator.xor, range(6))

1

In [13]:
#zip()
from itertools import zip_longest
list(zip_longest(range(3), 'ABC', [0.0, 1.1, 2.2, 3.3], fillvalue=-1))

[(0, 'A', 0.0), (1, 'B', 1.1), (2, 'C', 2.2), (-1, -1, 3.3)]

### Vector Take #5: Formatting
